In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
%matplotlib inline

In [46]:
results = pd.read_csv('data/fifa/international_results.csv')
results['date'] = pd.to_datetime(results['date'], utc=True)

# restrict dates
earliest_date = '2010-01-01'
latest_date = '2018-06-14'
results = results[(results['date'] > earliest_date) & (results['date'] < latest_date)]
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
31700,2010-01-02 00:00:00+00:00,Iran,Korea DPR,1,0,Friendly,Doha,Qatar,True
31701,2010-01-02 00:00:00+00:00,Qatar,Mali,0,0,Friendly,Doha,Qatar,False
31702,2010-01-02 00:00:00+00:00,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True
31703,2010-01-02 00:00:00+00:00,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False
31704,2010-01-03 00:00:00+00:00,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True


In [47]:
ratings = pd.read_csv('data/team_stats_final.csv')
ratings['date'] = pd.to_datetime(ratings['date'])

# restrict dates
ratings = ratings[(ratings['date'] > earliest_date) & (ratings['date'] < latest_date)]
ratings.head()

,team,date,attack,defence,full_age,midfield,overall,prestige,start_age,bup_speed,...,cc_passing,cc_crossing,cc_shooting,d_pressure,d_aggresion,d_width,wage_euros_thousands,value_euros_millions,growth,goalkeeeper_overall
1364,Brazil,2018-05-28,86.0,85.0,27.09,83.0,85.0,10.0,26.73,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.500000,1.826087,84.0
1365,England,2018-05-28,84.0,81.0,24.65,81.0,82.0,8.0,25.00,49.0,...,55.0,41.0,49.0,43.0,55.0,49.0,100.391304,21.065217,3.608696,78.0
1366,Italy,2018-05-28,85.0,85.0,26.04,81.0,83.0,9.0,28.45,73.0,...,64.0,80.0,78.0,29.0,33.0,32.0,91.521739,24.869565,3.173913,89.0
1367,Spain,2018-05-28,84.0,86.0,27.04,86.0,86.0,9.0,28.18,32.0,...,27.0,32.0,20.0,75.0,62.0,52.0,168.695652,39.239130,1.826087,91.0
1368,France,2018-05-28,83.0,82.0,25.65,85.0,84.0,9.0,25.18,35.0,...,24.0,53.0,35.0,47.0,47.0,67.0,110.782609,31.304348,3.173913,88.0


We now have to find the closest match in date between our FIFA team ratings and the match data that we have.

In [48]:
dates = pd.Series(ratings.date.unique())
teams = ratings.team.unique()

In [49]:
# check that we found the corresponding team from team ratings in results
for t in teams:
    try:
        assert np.sum(results.home_team.unique() == t) == 1
    except:
        print('Cannot Find {}'.format(t))

for t in teams:
    try:
        assert np.sum(results.away_team.unique() == t) == 1
    except:
        print('Cannot Find {}'.format(t))

Cannot Find Republic of Ireland
Cannot Find United States
Cannot Find China PR
Cannot Find Côte d'Ivoire
Cannot Find Republic of Ireland
Cannot Find United States
Cannot Find China PR
Cannot Find Côte d'Ivoire


Before we proceed with anything else, let's check that country names are spelled similarly. We find that 'Republic of Ireland', 'United States', 'China PR', and 'Côte d'Ivoire' might be spelled differently.

In [50]:
ratings.replace('Republic of Ireland', 'Ireland', inplace=True)
ratings.replace('United States', 'USA', inplace=True)
ratings.replace('China PR', 'China', inplace=True)
ratings.replace("Côte d'Ivoire", 'Ivory Coast', inplace=True)

Now we finally have two datasets of teams spelled in the same way.

In [51]:
# find closest date that we have data on
def get_latest_date(match_date, dates):
    match_date = match_date.to_datetime64()
    return dates[match_date >  dates].max()

# results['closest_date'] = results.apply(lambda i: min(dates, key=lambda d: abs(d-i.date)), axis=1)
results['closest_date'] = results['date'].apply(get_latest_date, dates = dates)


In [52]:
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,closest_date
39601,2018-06-10 00:00:00+00:00,Austria,Brazil,0,3,Friendly,Vienna,Austria,False,2018-06-07
39602,2018-06-11 00:00:00+00:00,Korea Republic,Senegal,0,2,Friendly,Grödig,Austria,True,2018-06-07
39603,2018-06-11 00:00:00+00:00,Belgium,Costa Rica,4,1,Friendly,Brussels,Belgium,False,2018-06-07
39604,2018-06-12 00:00:00+00:00,Japan,Paraguay,4,2,Friendly,Innsbruck,Austria,True,2018-06-11
39605,2018-06-12 00:00:00+00:00,Poland,Lithuania,4,0,Friendly,Warsaw,Poland,False,2018-06-11


Now that we have the closest matching data for both of our datasets, we can merge on team and date. Note that we have merge twice in order to account for both teams in the results table.

In [53]:
# merge home team with their closest ratings
results_ratings1 = results.merge(ratings, how='inner', 
                                left_on=['closest_date', 'home_team'],
                                right_on=['date', 'team'])

results_ratings2 = results_ratings1.merge(ratings, how='inner',
                                       left_on=['closest_date', 'away_team'],
                                       right_on=['date', 'team'])

In [54]:
results_ratings2.shape

(1871, 54)

We find that we have 1871 observations that can still be used after matching team ratings with the results dataframe with an inner merge. We now clean up the merged dataframe slightly.

In [55]:
# response variable
score_diff = results_ratings2['home_score'] - results_ratings2['away_score']
results_ratings2['home_win'] = [0 if score < 0 else 1 if score > 0 else 2 for score in score_diff]

# drop useless columns
results_ratings2.drop(['home_team', 'away_team', 'tournament', 
                       'city', 'country', 'neutral', 'closest_date',
                      'team_x', 'date_y', 'team_y', 'date',
                      'home_score', 'away_score', 'date_x'], axis=1, inplace=True)

# reorder columns
results_ratings2.sort_index(axis=1, inplace=True)
results_ratings2.head()

,attack_x,attack_y,bup_dribbling_x,bup_dribbling_y,bup_passing_x,bup_passing_y,bup_speed_x,bup_speed_y,cc_crossing_x,cc_crossing_y,...,overall_x,overall_y,prestige_x,prestige_y,start_age_x,start_age_y,value_euros_millions_x,value_euros_millions_y,wage_euros_thousands_x,wage_euros_thousands_y
0,73.0,67.0,0.0,52.0,45.0,52.0,65.0,50.0,55.0,72.0,...,77.0,67.0,15.0,1.0,27.09,25.18,0.0,0.275,0.0,0.869565
1,73.0,60.0,0.0,0.0,45.0,70.0,65.0,70.0,55.0,70.0,...,77.0,62.0,15.0,6.0,27.09,29.18,0.0,0.000,0.0,0.000000
2,71.0,60.0,0.0,0.0,70.0,70.0,70.0,70.0,60.0,70.0,...,75.0,62.0,11.0,6.0,30.36,29.18,0.0,0.000,0.0,0.000000
3,83.0,60.0,0.0,0.0,30.0,70.0,67.0,70.0,60.0,70.0,...,83.0,62.0,19.0,6.0,28.91,29.18,0.0,0.000,0.0,0.000000
4,70.0,60.0,0.0,0.0,70.0,70.0,70.0,70.0,65.0,70.0,...,70.0,62.0,5.0,6.0,25.64,29.18,0.0,0.000,0.0,0.000000


Our final training data is basically a difference in the various team ratings. Now let's clean up the dataframe for our training.

In [56]:
ratings_base = results_ratings2.loc[:,['home_win']]
ratings_base.head()

,home_win
0,1
1,1
2,1
3,2
4,1


In [57]:
diff_ratings = results_ratings2.drop('home_win', axis=1)
diff_ratings = diff_ratings.diff(axis=1)
ncol = diff_ratings.shape[1]

# we only want every alternate column
diff_ratings = diff_ratings.iloc[:,list(np.arange(1,ncol, 2))]

# we want our statistics to be from perspective of home team
diff_ratings = diff_ratings*-1
diff_ratings.head()

,attack_y,bup_dribbling_y,bup_passing_y,bup_speed_y,cc_crossing_y,cc_passing_y,cc_shooting_y,d_aggresion_y,d_pressure_y,d_width_y,defence_y,full_age_y,goalkeeeper_overall_y,growth_y,midfield_y,overall_y,prestige_y,start_age_y,value_euros_millions_y,wage_euros_thousands_y
0,6.0,-52.0,-7.0,15.0,-17.0,12.0,1.0,27.0,18.0,-4.0,9.0,1.46,13.0,2.208696,6.0,10.0,14.0,1.91,-0.275,-0.869565
1,13.0,-0.0,-25.0,-5.0,-15.0,-5.0,-5.0,-10.0,-5.0,25.0,16.0,0.23,19.0,-1.666667,17.0,15.0,9.0,-2.09,-0.000,-0.000000
2,11.0,-0.0,-0.0,-0.0,-10.0,-0.0,-0.0,-0.0,-20.0,15.0,10.0,1.53,19.0,-1.466667,14.0,13.0,5.0,1.18,-0.000,-0.000000
3,23.0,-0.0,-40.0,-3.0,-10.0,-15.0,-30.0,-40.0,-40.0,15.0,22.0,1.20,28.0,-3.433333,24.0,21.0,13.0,-0.27,-0.000,-0.000000
4,10.0,-0.0,-0.0,-0.0,-5.0,-15.0,-0.0,-0.0,-20.0,15.0,9.0,-0.30,18.0,0.300000,10.0,8.0,-1.0,-3.54,-0.000,-0.000000


In [58]:
columns = ['attack_diff', 'bup_dribbling_diff', 'bup_passing_diff', 'bup_speed_diff',
          'cc_crossing_diff', 'cc_passing_diff', 'cc_shooting_diff', 'd_aggresion_diff',
          'd_pressure_diff', 'd_width_diff', 'defence_diff', 'full_age_diff', 
          'goalkeeper_overall_diff', 'growth_diff', 'midfield_diff', 'overall_diff',
          'prestige_diff', 'start_age_diff', 'avg_value_euros_millions_diff',
          'avg_wage_euros_thousands_diff']

diff_ratings.columns = columns

# compile datframe
train = pd.concat([ratings_base, diff_ratings], axis=1)
train.head()

,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,cc_crossing_diff,cc_passing_diff,cc_shooting_diff,d_aggresion_diff,d_pressure_diff,...,defence_diff,full_age_diff,goalkeeper_overall_diff,growth_diff,midfield_diff,overall_diff,prestige_diff,start_age_diff,avg_value_euros_millions_diff,avg_wage_euros_thousands_diff
0,1,6.0,-52.0,-7.0,15.0,-17.0,12.0,1.0,27.0,18.0,...,9.0,1.46,13.0,2.208696,6.0,10.0,14.0,1.91,-0.275,-0.869565
1,1,13.0,-0.0,-25.0,-5.0,-15.0,-5.0,-5.0,-10.0,-5.0,...,16.0,0.23,19.0,-1.666667,17.0,15.0,9.0,-2.09,-0.000,-0.000000
2,1,11.0,-0.0,-0.0,-0.0,-10.0,-0.0,-0.0,-0.0,-20.0,...,10.0,1.53,19.0,-1.466667,14.0,13.0,5.0,1.18,-0.000,-0.000000
3,2,23.0,-0.0,-40.0,-3.0,-10.0,-15.0,-30.0,-40.0,-40.0,...,22.0,1.20,28.0,-3.433333,24.0,21.0,13.0,-0.27,-0.000,-0.000000
4,1,10.0,-0.0,-0.0,-0.0,-5.0,-15.0,-0.0,-0.0,-20.0,...,9.0,-0.30,18.0,0.300000,10.0,8.0,-1.0,-3.54,-0.000,-0.000000


In [59]:
# save to csv
train.to_csv('data/train_team.csv', index = False)

We now create our test set with actual world cup data.

In [60]:
ratings_wc = pd.read_csv('data/team_stats_final.csv')
ratings_wc['date'] = pd.to_datetime(ratings_wc['date'], utc=True)

# restrict dates
latest_date = '2018-06-14'
wc_start = '2018-06-16' # first WC rating
wc_end = '2018-07-15'
# restrict dates
ratings_wc = ratings_wc[(ratings_wc['date'] >= wc_start) & (ratings_wc['date'] <= wc_end)]

ratings_wc.head()

,team,date,attack,defence,full_age,midfield,overall,prestige,start_age,bup_speed,...,cc_passing,cc_crossing,cc_shooting,d_pressure,d_aggresion,d_width,wage_euros_thousands,value_euros_millions,growth,goalkeeeper_overall
0,Brazil,2018-07-15 00:00:00+00:00,87.0,84.0,27.43,86.0,86.0,10.0,27.55,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,0.0,0.0,1.62500,84.0
1,England,2018-07-15 00:00:00+00:00,83.0,80.0,25.88,81.0,81.0,8.0,24.64,36.0,...,31.0,41.0,41.0,43.0,55.0,49.0,0.0,0.0,2.57500,80.0
2,Italy,2018-07-15 00:00:00+00:00,81.0,82.0,25.96,81.0,82.0,9.0,27.36,73.0,...,64.0,80.0,78.0,29.0,26.0,32.0,0.0,0.0,3.26087,88.0
3,Spain,2018-07-15 00:00:00+00:00,84.0,85.0,27.18,86.0,85.0,9.0,27.27,32.0,...,27.0,32.0,20.0,75.0,62.0,52.0,0.0,0.0,1.77500,91.0
4,France,2018-07-15 00:00:00+00:00,86.0,81.0,25.05,84.0,85.0,9.0,24.82,35.0,...,24.0,53.0,35.0,47.0,47.0,67.0,0.0,0.0,3.45000,87.0


In [61]:
dates_wc = ratings_wc.date.unique()
teams_wc = ratings_wc.team.unique()

In [106]:
#https://gitlab.com/djh_or/2018-world-cup-stats/blob/master/world_cup_2018_stats.csv
results_wc = pd.read_csv("data/world_cup_2018_stats.csv")
results_wc.head()

,Game,Group,Team,Opponent,Home/Away,Score,WDL,Pens?,Goals For,Goals Against,...,Passes Completed,Distance Covered km,Balls recovered,Tackles,Blocks,Clearances,Yellow cards,Red Cards,Second Yellow Card leading to Red Card,Fouls Committed
0,1,A,Russia,Saudi Arabia,home,5-0,W,NaN,5,0,...,240,118,53,9,3,19,1,0,0,22
1,1,A,Saudi Arabia,Russia,away,5-0,L,NaN,0,5,...,442,105,48,16,3,31,1,0,0,10
2,2,A,Egypt,Uruguay,home,0-1,L,NaN,0,1,...,308,112,57,12,4,32,2,0,0,12
3,2,A,Uruguay,Egypt,away,0-1,W,NaN,1,0,...,508,111,54,8,2,22,0,0,0,6
4,3,B,Morocco,IR Iran,home,0-1,L,NaN,0,1,...,371,101,38,9,1,16,1,0,0,22


Before we proceed, let's ensure that our country names match up similarly.

In [107]:
# check that we found the corresponding team from team ratings in results
# but now we reverse it such that we try to find a rating for each wc team
for t in results_wc.Team.unique():
    try:
        assert np.sum(teams_wc == t) == 1
    except:
        print('Cannot Find {}'.format(t))


Cannot Find IR Iran


In [108]:
ratings_wc.replace('Iran', 'IR Iran', inplace=True)

In [109]:
tournament_round = results_wc.Group
results_wc = results_wc.loc[:,['Team', 'Opponent', 'WDL']]
results_wc['home_win'] = [0 if score == 'L' else 1 if score == 'W' else 2 for score in results_wc.WDL]
results_wc.drop('WDL', axis=1, inplace=True)
results_wc.tail()

,Team,Opponent,home_win
123,England,Croatia,0
124,Belgium,England,1
125,England,Belgium,0
126,France,Croatia,1
127,Croatia,France,0


Now we can match the ratings data to our test dataset as well. We note that there might be some value in using the updated FIFA ratings at each time of the match. However, for simplicity, let's just assume that teams have the same rating throughout the tournament. We will take the rating at the start of the tournament.

In [110]:
ratings_wc_start = ratings_wc[ratings_wc['date'] == wc_start]

# merge ratings with results table
results_ratings_wc1 = results_wc.merge(ratings_wc_start, how='left',
                                     left_on='Team', right_on='team')
results_ratings_wc2 = results_ratings_wc1.merge(ratings_wc_start, how='left',
                                             left_on='Opponent', right_on='team')



In [111]:
# check that every team has a rating
assert results_ratings_wc2.shape[0] == results_wc.shape[0]

In [112]:
# drop useless columns
results_ratings_wc2.drop(['Team', 'Opponent', 'team_x', 'team_y', 'date_y', 'date_x'], 
                         axis=1, inplace=True)

# reorder columns
results_ratings_wc2.sort_index(axis=1, inplace=True)
results_ratings_wc2.head()

,attack_x,attack_y,bup_dribbling_x,bup_dribbling_y,bup_passing_x,bup_passing_y,bup_speed_x,bup_speed_y,cc_crossing_x,cc_crossing_y,...,overall_x,overall_y,prestige_x,prestige_y,start_age_x,start_age_y,value_euros_millions_x,value_euros_millions_y,wage_euros_thousands_x,wage_euros_thousands_y
0,80.0,71.0,77.0,68.0,49.0,66.0,50.0,69.0,37.0,48.0,...,79.0,72.0,6.0,4.0,27.82,28.00,0.0,0.0,0.0,0.0
1,71.0,80.0,68.0,77.0,66.0,49.0,69.0,50.0,48.0,37.0,...,72.0,79.0,4.0,6.0,28.00,27.82,0.0,0.0,0.0,0.0
2,72.0,86.0,34.0,42.0,49.0,36.0,52.0,38.0,64.0,43.0,...,76.0,80.0,5.0,7.0,27.64,26.09,0.0,0.0,0.0,0.0
3,86.0,72.0,42.0,34.0,36.0,49.0,38.0,52.0,43.0,64.0,...,80.0,76.0,7.0,5.0,26.09,27.64,0.0,0.0,0.0,0.0
4,72.0,79.0,52.0,67.0,38.0,69.0,38.0,69.0,58.0,37.0,...,76.0,74.0,3.0,3.0,26.55,26.27,0.0,0.0,0.0,0.0


In [113]:
ratings_base_wc = results_ratings_wc2.loc[:,['home_win']]
ratings_base_wc.head()

,home_win
0,1
1,0
2,0
3,1
4,0


In [114]:
diff_ratings_wc = results_ratings_wc2.drop('home_win', axis=1)
diff_ratings_wc = diff_ratings_wc.diff(axis=1)
ncol_wc = diff_ratings_wc.shape[1]

# we only want every alternate column
diff_ratings_wc = diff_ratings_wc.iloc[:,list(np.arange(1,ncol_wc, 2))]

# we want our statistics to be from perspective of home teams
diff_ratings_wc = diff_ratings_wc*-1
diff_ratings_wc.head()

,attack_y,bup_dribbling_y,bup_passing_y,bup_speed_y,cc_crossing_y,cc_passing_y,cc_shooting_y,d_aggresion_y,d_pressure_y,d_width_y,defence_y,full_age_y,goalkeeeper_overall_y,growth_y,midfield_y,overall_y,prestige_y,start_age_y,value_euros_millions_y,wage_euros_thousands_y
0,9.0,9.0,-17.0,-19.0,-11.0,-9.0,-27.0,1.0,-0.0,-17.0,6.0,-0.13,11.0,-0.000,6.0,7.0,2.0,-0.18,-0.0,-0.0
1,-9.0,-9.0,17.0,19.0,11.0,9.0,27.0,-1.0,-0.0,17.0,-6.0,0.13,-11.0,-0.000,-6.0,-7.0,-2.0,0.18,-0.0,-0.0
2,-14.0,-8.0,13.0,14.0,21.0,25.0,-12.0,-9.0,-0.0,-30.0,-5.0,0.30,-15.0,-0.425,-0.0,-4.0,-2.0,1.55,-0.0,-0.0
3,14.0,8.0,-13.0,-14.0,-21.0,-25.0,12.0,9.0,-0.0,30.0,5.0,-0.30,15.0,0.425,-0.0,4.0,2.0,-1.55,-0.0,-0.0
4,-7.0,-15.0,-31.0,-31.0,21.0,-9.0,-9.0,9.0,17.0,46.0,7.0,-0.30,-3.0,-0.175,6.0,2.0,-0.0,0.28,-0.0,-0.0


In [115]:
diff_ratings_wc.columns = columns

# compile datframe
test = pd.concat([ratings_base_wc, diff_ratings_wc], axis=1)
test.head()

,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,cc_crossing_diff,cc_passing_diff,cc_shooting_diff,d_aggresion_diff,d_pressure_diff,...,defence_diff,full_age_diff,goalkeeper_overall_diff,growth_diff,midfield_diff,overall_diff,prestige_diff,start_age_diff,avg_value_euros_millions_diff,avg_wage_euros_thousands_diff
0,1,9.0,9.0,-17.0,-19.0,-11.0,-9.0,-27.0,1.0,-0.0,...,6.0,-0.13,11.0,-0.000,6.0,7.0,2.0,-0.18,-0.0,-0.0
1,0,-9.0,-9.0,17.0,19.0,11.0,9.0,27.0,-1.0,-0.0,...,-6.0,0.13,-11.0,-0.000,-6.0,-7.0,-2.0,0.18,-0.0,-0.0
2,0,-14.0,-8.0,13.0,14.0,21.0,25.0,-12.0,-9.0,-0.0,...,-5.0,0.30,-15.0,-0.425,-0.0,-4.0,-2.0,1.55,-0.0,-0.0
3,1,14.0,8.0,-13.0,-14.0,-21.0,-25.0,12.0,9.0,-0.0,...,5.0,-0.30,15.0,0.425,-0.0,4.0,2.0,-1.55,-0.0,-0.0
4,0,-7.0,-15.0,-31.0,-31.0,21.0,-9.0,-9.0,9.0,17.0,...,7.0,-0.30,-3.0,-0.175,6.0,2.0,-0.0,0.28,-0.0,-0.0


In [118]:
test['Group'] = tournament_round

In [119]:
#get rid of even rows since same match as odd rows.
test = test[test.index % 2 == 0]

In [120]:
# save to csv
test.to_csv('data/test_team.csv', index = False)

In [121]:
test

,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,cc_crossing_diff,cc_passing_diff,cc_shooting_diff,d_aggresion_diff,d_pressure_diff,...,full_age_diff,goalkeeper_overall_diff,growth_diff,midfield_diff,overall_diff,prestige_diff,start_age_diff,avg_value_euros_millions_diff,avg_wage_euros_thousands_diff,Group
0,1,9.0,9.0,-17.0,-19.0,-11.0,-9.0,-27.0,1.0,-0.0,...,-0.13,11.0,-0.000000,6.0,7.0,2.0,-0.18,-0.0,-0.0,A
2,0,-14.0,-8.0,13.0,14.0,21.0,25.0,-12.0,-9.0,-0.0,...,0.30,-15.0,-0.425000,-0.0,-4.0,-2.0,1.55,-0.0,-0.0,A
4,0,-7.0,-15.0,-31.0,-31.0,21.0,-9.0,-9.0,9.0,17.0,...,-0.30,-3.0,-0.175000,6.0,2.0,-0.0,0.28,-0.0,-0.0,B
6,2,-0.0,27.0,22.0,19.0,26.0,27.0,42.0,-4.0,-9.0,...,-0.50,-6.0,1.000000,-3.0,-1.0,-1.0,-1.54,-0.0,-0.0,B
8,1,16.0,6.0,-19.0,-14.0,7.0,-28.0,-31.0,-5.0,4.0,...,-1.68,11.0,1.050000,12.0,13.0,4.0,-3.27,-0.0,-0.0,C
10,2,8.0,-6.0,-12.0,-13.0,-20.0,-34.0,-8.0,-4.0,33.0,...,1.08,4.0,-1.098684,7.0,9.0,7.0,-1.10,-0.0,-0.0,D
12,0,2.0,-2.0,-4.0,3.0,38.0,-22.0,11.0,8.0,-2.0,...,0.83,-7.0,-0.122436,-3.0,-2.0,-2.0,0.82,-0.0,-0.0,C
14,1,6.0,-17.0,-20.0,-17.0,3.0,-27.0,-16.0,-23.0,-9.0,...,1.23,15.0,-1.115789,7.0,5.0,2.0,3.82,-0.0,-0.0,D
16,0,-5.0,4.0,-0.0,-1.0,44.0,-6.0,-5.0,-6.0,-1.0,...,3.13,12.0,-2.225000,-6.0,-5.0,-3.0,0.91,-0.0,-0.0,E
18,0,5.0,30.0,-6.0,-12.0,-20.0,-20.0,-34.0,-0.0,-2.0,...,-2.70,12.0,0.825000,6.0,7.0,3.0,-0.64,-0.0,-0.0,F


In [122]:
train

,home_win,attack_diff,bup_dribbling_diff,bup_passing_diff,bup_speed_diff,cc_crossing_diff,cc_passing_diff,cc_shooting_diff,d_aggresion_diff,d_pressure_diff,...,defence_diff,full_age_diff,goalkeeper_overall_diff,growth_diff,midfield_diff,overall_diff,prestige_diff,start_age_diff,avg_value_euros_millions_diff,avg_wage_euros_thousands_diff
0,1,6.0,-52.0,-7.0,15.0,-17.0,12.0,1.0,27.0,18.0,...,9.0,1.46,13.0,2.208696,6.0,10.0,14.0,1.91,-0.275000,-0.869565
1,1,13.0,-0.0,-25.0,-5.0,-15.0,-5.0,-5.0,-10.0,-5.0,...,16.0,0.23,19.0,-1.666667,17.0,15.0,9.0,-2.09,-0.000000,-0.000000
2,1,11.0,-0.0,-0.0,-0.0,-10.0,-0.0,-0.0,-0.0,-20.0,...,10.0,1.53,19.0,-1.466667,14.0,13.0,5.0,1.18,-0.000000,-0.000000
3,2,23.0,-0.0,-40.0,-3.0,-10.0,-15.0,-30.0,-40.0,-40.0,...,22.0,1.20,28.0,-3.433333,24.0,21.0,13.0,-0.27,-0.000000,-0.000000
4,1,10.0,-0.0,-0.0,-0.0,-5.0,-15.0,-0.0,-0.0,-20.0,...,9.0,-0.30,18.0,0.300000,10.0,8.0,-1.0,-3.54,-0.000000,-0.000000
5,2,15.0,-0.0,-30.0,-5.0,-30.0,-5.0,-0.0,-25.0,-35.0,...,15.0,1.53,16.0,-3.166667,13.0,13.0,2.0,-1.09,-0.000000,-0.000000
6,2,-2.0,-49.0,-6.0,13.0,6.0,-2.0,32.0,13.0,18.0,...,3.0,-2.06,10.0,4.034783,1.0,5.0,12.0,-3.46,-2.938043,-14.000000
7,2,6.0,-0.0,-0.0,-5.0,10.0,-0.0,-5.0,-5.0,-0.0,...,9.0,-1.63,14.0,-1.300000,5.0,5.0,6.0,-1.09,-0.000000,-0.000000
8,1,-0.0,-0.0,15.0,-0.0,20.0,-0.0,-0.0,-5.0,-0.0,...,-0.0,-2.26,8.0,-2.600000,-0.0,-2.0,1.0,-1.63,-0.000000,-0.000000
9,1,-4.0,-53.0,-6.0,14.0,2.0,6.0,17.0,1.0,17.0,...,-2.0,0.15,10.0,4.025000,-3.0,1.0,8.0,1.91,-0.000000,-0.000000
